### General Axissymmetric element
Linear tri, linear and quadratic quad elements for axisymmetric analysis.

Here is a step-by-step outline of the numerical procedure for this type of analysis.

***

### ## 1. Pre-processing

* **Create 2D Mesh:** First, create a standard 2D finite element mesh of the structure's cross-section in the $r-z$ plane.
* **Decompose the Load:** Analyze your applied external load and decompose it into a Fourier series around the circumference ($\theta$). This gives you the load amplitude functions (e.g., $p_{rn}(r, z)$) for each required harmonic $n$.
* **Compute Fundamental Matrices:** For each element in the mesh, compute the three fundamental, harmonic-independent stiffness matrices: $[K_0]$, $[K_1]$, and $[K_2]$. This is done only once.

***

### ## 2. Analysis Loop 🔄

You then loop through each harmonic $n$ that is significant for your decomposed load (e.g., $n=0, 1, 2, ... N$).

**For each harmonic $n$ in the loop:**

1.  **Initialize Global Matrices:** Create empty global matrices for the stiffness $[K_n]$ and force vector $\{F_n\}$.
2.  **Element Loop:** For each element in the 2D mesh:
    * **Assemble Element Stiffness:** Calculate the element stiffness matrix for the current harmonic using the pre-computed base matrices: $[k_e]_n = [K_0] + n[K_1] + n^2[K_2]$.
    * **Calculate Element Forces:** Calculate the element's nodal force vector $\{F_n\}_e$ by integrating the load amplitudes for harmonic $n$ with the element's shape functions.
    * **Assemble into Global:** Add the element matrices $[k_e]_n$ and $\{F_n\}_e$ into the appropriate positions in the global matrices $[K_n]$ and $\{F_n\}$.
3.  **Apply Boundary Conditions:** Modify the global system to account for any prescribed zero or non-zero displacements for the current harmonic $n$.
4.  **Solve System:** Solve the linear system of equations $[K_n]\{u_n\} = \{F_n\}$ to find the vector of nodal displacement amplitudes $\{u_n\}$ for this harmonic.
5.  **Store Results:** Save the resulting displacement amplitudes $\{u_n\}$ and, if desired, calculate and save the stress amplitudes for this harmonic.

***

### ## 3. Post-processing

* **Reconstruct Solution:** After the analysis loop is complete, the final 3D displacement and stress fields are obtained by summing (superimposing) the results from each individual harmonic at any desired point $(r, \theta, z)$.

In [1]:
import copy
import time
from typing import Callable

import numpy as np
import sympy as sp
sp.init_printing()

### The physical properties of the element

In [2]:
ND = 3  # number of degrees of freedom per node
NNODE = 4  # number of nodes
if NNODE == 3:
    NGS = [1, 3]  # number of Gauss points, can be 1, 3
elif NNODE == 4:
    NGS = [1, 2]  # number of Gauss points, can be 1, 2, 3, 4, 5, or 6
elif NNODE == 8:
    NGS = [1, 2]  # number of Gauss points, can be 1, 2, 3, 4, 5, or 6
else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

### The tri element

```
    ^
    |t
  3 |
  | \
  | |\
  | |-\--->s
  |    \
  1-----2
```

### The linear quad element

Node numbering is CCW, starting at ``(-1, -1)``:
```
     ^
     |t
     |
  4--|--3
  |  |  |
  |  |--|--->s
  |     |
  1-----2
```

### The quadratic quad element

Node numbering is CCW, starting at ``(-1, -1)``:
```
      ^
      |t
      |
  4---|7---3
  |   |    |
  8   |----6--->s
  |        |
  1---5----2

      Z
      ^
      |
  4---7---3
  |       |
  8       6  --> R
  |       |
  1---5---2

```

It has 3 degrees of freedom each node:
- u: horizontal displacement
- v: vertical displacement
- theta: circumferential displacement

The DOFs are ordered as follows: u, theta, v.


In [3]:
### The physical properties of the element
E = 200e9  # Young's modulus in Pa
nu = 0.30  # Poisson's ratio

# Define symbolic physical coordinates
r1, z1, th1, r2, z2, th2, r3, z3, th3, r4, z4, th4, r5, z5, th5, r6, z6, th6, r7, z7, th7, r8, z8, th8 = sp.symbols('r1 z1 th1 r2 z2 th2 r3 z3 th3 r4 z4 th4 r5 z5 th5 r6 z6 th6 r7 z7 th7 r8 z8 th8')

# Define symbolic variables. Capital letters R, Z are the symbols for the variables r, z
R, Z, TH = sp.symbols('R Z TH')  # radial [m], vertical [m] and circumferential [°] coordinates

if NNODE == 3:
    # Example coordinates for the nodes
    # physical_coords = {'P1': (0, 0, 0),
    #                    'P2': (1, 0, 0),
    #                    'P3': (0, 1, 0)}  # example coordinates for the nodes
    # physical_coords = {'P1': (3, 0, 0),
    #                    'P2': (4, 0, 0),
    #                    'P3': (3, 0, 1)}  # example coordinates for the nodes

    physical_coords = {'P1': (10000, 0, 0),
                       'P2': (10020, 0, 200),
                       'P3': (10000, 0, 1000)}  # example coordinates for the nodes

    midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0]) / 3, \
                (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1]) / 3, \
        (physical_coords['P1'][2] + physical_coords['P2'][2] + physical_coords['P3'][2]) / 3

    physical_coords_map = {'r1': physical_coords['P1'][0], 'th1': physical_coords['P1'][1], 'z1': physical_coords['P1'][2],
                           'r2': physical_coords['P2'][0], 'th2': physical_coords['P2'][1], 'z2': physical_coords['P2'][2],
                           'r3': physical_coords['P3'][0], 'th3': physical_coords['P3'][1], 'z3': physical_coords['P3'][2],}
    # Define the physical nodes as a matrix so it can be used a symbolic evaluation
    physical_nodes = sp.Matrix([[r1, th1, z1],
                                [r2, th2, z2],
                                [r3, th3, z3]])


elif NNODE == 4:

    # physical_coords = {'P1': (3, 0, 0),
    #                    'P2': (4, 0, 0),
    #                    'P3': (4, 0, 1),
    #                    'P4': (3, 0, 1),
    #                    }  # example coordinates for the nodes
    # physical_coords = {'P1': (0.11, 0, 0),
    #                    'P2': (0.20, 0, 0),
    #                    'P3': (0.20, 0, 0.04),
    #                    'P4': (0.11, 0, 0.04),
    #                    }  # example coordinates for the nodes

    # cylinder, di=1000mm, h=1000mm, t=10 mm
    physical_coords = {'P1': (1000, 0, 0),
                       'P2': (1020, 0, 0),
                       'P3': (1020, 0, 1000),
                       'P4': (1000, 0, 1000),
                       }  # example coordinates for the nodes

    midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0] + physical_coords['P4'][0]) / 4, \
                (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1] + physical_coords['P4'][1]) / 4, \
        (physical_coords['P1'][2] + physical_coords['P2'][2] + physical_coords['P3'][2] + physical_coords['P4'][2]) / 4

    physical_coords_map = {'r1': physical_coords['P1'][0], 'th1': physical_coords['P1'][1], 'z1': physical_coords['P1'][2],
                           'r2': physical_coords['P2'][0], 'th2': physical_coords['P2'][1], 'z2': physical_coords['P2'][2],
                           'r3': physical_coords['P3'][0], 'th3': physical_coords['P3'][1], 'z3': physical_coords['P3'][2],
                           'r4': physical_coords['P4'][0], 'th4': physical_coords['P4'][1], 'z4': physical_coords['P4'][2],
                           }
    # Define the physical nodes as a matrix so it can be used a symbolic evaluation
    physical_nodes = sp.Matrix([[r1, th1, z1],
                                [r2, th2, z2],
                                [r3, th3, z3],
                                [r4, th4, z4]])
elif NNODE == 8:

    def midpoint(p1, p2):
        return (p1[0] + p2[0]) / 2, 0, (p1[2] + p2[2]) / 2

    # physical_coords = {'P1': (3, 0, 0),
    #                    'P2': (4, 0, 0),
    #                    'P3': (4, 0, 1),
    #                    'P4': (3, 0, 1),
    #                    }  # example coordinates for the nodes
    # physical_coords = {'P1': (0.11, 0, 0),
    #                    'P2': (0.20, 0, 0),
    #                    'P3': (0.20, 0, 0.04),
    #                    'P4': (0.11, 0, 0.04),
    #                    }  # example coordinates for the nodes

    # cylinder, di=1000mm, h=1000mm, t=10 mm
    physical_coords = {'P1': (10000, 0, 0),
                       'P2': (10020, 0, 0),
                       'P3': (10020, 0, 1000),
                       'P4': (10000, 0, 1000),
                       }  # example coordinates for the nodes

    physical_coords['P5'] = midpoint(physical_coords['P1'], physical_coords['P2'])
    physical_coords['P6'] = midpoint(physical_coords['P2'], physical_coords['P3'])
    physical_coords['P7'] = midpoint(physical_coords['P3'], physical_coords['P4'])
    physical_coords['P8'] = midpoint(physical_coords['P4'], physical_coords['P1'])

    physical_coords_map = {'r1': physical_coords['P1'][0], 'th1': physical_coords['P1'][1], 'z1': physical_coords['P1'][2],
                           'r2': physical_coords['P2'][0], 'th2': physical_coords['P2'][1], 'z2': physical_coords['P2'][2],
                           'r3': physical_coords['P3'][0], 'th3': physical_coords['P3'][1], 'z3': physical_coords['P3'][2],
                           'r4': physical_coords['P4'][0], 'th4': physical_coords['P4'][1], 'z4': physical_coords['P4'][2],
                           'r5': physical_coords['P5'][0], 'th5': physical_coords['P5'][1], 'z5': physical_coords['P5'][2],
                           'r6': physical_coords['P6'][0], 'th6': physical_coords['P6'][1], 'z6': physical_coords['P6'][2],
                           'r7': physical_coords['P7'][0], 'th7': physical_coords['P7'][1], 'z7': physical_coords['P7'][2],
                           'r8': physical_coords['P8'][0], 'th8': physical_coords['P8'][1], 'z8': physical_coords['P8'][2],
                           }

    # Define the physical nodes as a matrix so it can be used a symbolic evaluation
    physical_nodes = sp.Matrix([[r1, th1, z1],
                                [r2, th2, z2],
                                [r3, th3, z3],
                                [r4, th4, z4],
                                [r5, th5, z5],
                                [r6, th6, z6],
                                [r7, th7, z7],
                                [r8, th8, z8],

                                ])

else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

import pprint
pprint.pprint(physical_coords)

{'P1': (1000, 0, 0),
 'P2': (1020, 0, 0),
 'P3': (1020, 0, 1000),
 'P4': (1000, 0, 1000)}


In [4]:
# Define the shape functions symbolically
def shape_functions() -> sp.Matrix:
    """shape functions"""

    if NNODE == 3:
        return sp.Matrix([
            1 - R - Z,
            R,
            Z,
        ])
    elif NNODE == 4:
        return sp.Matrix([
            0.25 * (1 - R) * (1 - Z),
            0.25 * (1 + R) * (1 - Z),
            0.25 * (1 + R) * (1 + Z),
            0.25 * (1 - R) * (1 + Z),
        ])
    elif NNODE == 8:
        return sp.Matrix([
            0.25 * (1 - R) * (1 - Z) * (-R - Z -1),
            0.25 * (1 + R) * (1 - Z) * (R - Z -1),
            0.25 * (1 + R) * (1 + Z) * (R + Z -1),
            0.25 * (1 - R) * (1 + Z) * (-R + Z -1),

            0.5 * (1 - R ** 2) * (1 - Z),  # 5
            0.5 * (1 + R) * (1 - Z ** 2),  # 6
            0.5 * (1 - R ** 2) * (1 + Z),  # 7
            0.5 * (1 - R) * (1 - Z ** 2),  # 8

        ])
    else:
        raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")


# checking the shape functions.
if NNODE == 3:
    coords = (
        (0, 0),  # 1
        (1, 0),  # 2
        (0, 1)   # 3
    )
elif NNODE == 4:
    coords = (
        (-1, -1),  # 1
        (1, -1),  # 2
        (1, 1),  # 3
        (-1, 1)  # 4
    )
elif NNODE == 8:
    coords = (
    (-1, -1),  # 1
    (1, -1),  # 2
    (1, 1),  # 3
    (-1, 1),  # 4
    (0, -1),  # 5
    (1, 0),  # 6
    (0, 1),  # 7
    (-1, 0)  # 8
    )
else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

sh = shape_functions()  # N1, N2, N3, N4, N5, N6, N7, N8
for cindex, coord in enumerate(coords):
    s, z = coord
    for i in range(NNODE):
        if i != cindex:
            assert np.isclose(sh.subs({R: s, Z: z})[i], 0)
        else:
            assert np.isclose(sh.subs({R: s, Z: z})[i], 1)



def isoparametric_mapping() -> np.array:
    """
    Isoparametric mapping natural -> physical coordinates.

    :return: functions returning the physical coordinates
    """

    sh = shape_functions()  # shape functions with substituted values
    return sp.Matrix([sp.Matrix(sh).dot(physical_nodes[:, 0]), sp.Matrix(sh).dot(physical_nodes[:, 2])])


# Calculate the derivatives by r and z, symbolically
def shape_function_derivatives() -> np.array:
    """
    Shape function derivatives.
    [0] = dNi/dr, [1] = dNi/dz
    """
    sh = shape_functions()  # N1, N2, N3, N4
    dN_dr = []
    dN_dz = []

    for i in range(len(sh)):
        dN_dr.append(sp.diff(sh[i], R))
        dN_dz.append(sp.diff(sh[i], Z))

    return dN_dr, dN_dz


print()
print('The shape functions are:')
print(shape_functions())
print()
print('The derivatives are:')
derivatives = shape_function_derivatives()
for i, dN in enumerate(derivatives[0]):
    print(f'dN{i + 1}/ds = {dN}')
for i, dN in enumerate(derivatives[1]):
    print(f'dN{i + 1}/dt = {dN}')

print()
print('The isoparametric mapping is:')
x_map = isoparametric_mapping()[0]
y_map = isoparametric_mapping()[1]
print(f'x = {x_map}')
print(f'y = {y_map}')

# Substitute the symbolic coordinates with numerical values
st_values = {
    R: 0.3333,  # example value for s
    Z: 0.3333,  # example value for t
}
print()
print('The isoparametric mapping with coordinate values substituted is:')
x_map_substituted = x_map.subs(st_values)
y_map_substituted = y_map.subs(st_values)
print(f'x = {x_map_substituted}')
print(f'y = {y_map_substituted}')

print()
print('The derivatives of the shape functions by s are:')
print(derivatives[0])  # derivatives by s

print()
print('The derivatives of the shape functions by t are:')
print(derivatives[1])  # derivatives by t



The shape functions are:
Matrix([[(0.25 - 0.25*R)*(1 - Z)], [(1 - Z)*(0.25*R + 0.25)], [(0.25*R + 0.25)*(Z + 1)], [(0.25 - 0.25*R)*(Z + 1)]])

The derivatives are:
dN1/ds = 0.25*Z - 0.25
dN2/ds = 0.25 - 0.25*Z
dN3/ds = 0.25*Z + 0.25
dN4/ds = -0.25*Z - 0.25
dN1/dt = 0.25*R - 0.25
dN2/dt = -0.25*R - 0.25
dN3/dt = 0.25*R + 0.25
dN4/dt = 0.25 - 0.25*R

The isoparametric mapping is:
x = r1*(0.25 - 0.25*R)*(1 - Z) + r2*(1 - Z)*(0.25*R + 0.25) + r3*(0.25*R + 0.25)*(Z + 1) + r4*(0.25 - 0.25*R)*(Z + 1)
y = z1*(0.25 - 0.25*R)*(1 - Z) + z2*(1 - Z)*(0.25*R + 0.25) + z3*(0.25*R + 0.25)*(Z + 1) + z4*(0.25 - 0.25*R)*(Z + 1)

The isoparametric mapping with coordinate values substituted is:
x = 0.1111222225*r1 + 0.2222277775*r2 + 0.4444222225*r3 + 0.2222277775*r4
y = 0.1111222225*z1 + 0.2222277775*z2 + 0.4444222225*z3 + 0.2222277775*z4

The derivatives of the shape functions by s are:
[0.25*Z - 0.25, 0.25 - 0.25*Z, 0.25*Z + 0.25, -0.25*Z - 0.25]

The derivatives of the shape functions by t are:
[0.25*

### The shape function matrix

The shape function matrix $N$ is a matrix that contains the shape functions for each node. It has the structure:
$$
N = \begin{bmatrix}
N_i & 0 & 0 & ... & N_m & 0 & 0 \\
0 & N_i & 0 & ... & 0 & N_m & 0 \\
0 & 0 & N_i & ... & 0 & 0 & N_m \\
\end{bmatrix}
$$

In [5]:
def N_matrix() -> np.array:
    """
    shape function matrix N.

    """
    _N = shape_functions()  # the shape functions

    # preparing the return matrix
    _ret = sp.Matrix(ND, ND * NNODE, lambda i, j: 0)  # ND rows, ND * NNODE columns

    # filling the matrix
    for i in range(NNODE):
        for j in range(ND):
            _ret[j, j + ND * i] = _N[i]

    return _ret


_N = N_matrix()
print("The shape function matrix N is {}:".format(_N.shape))
print()
print('The shape function matrix N for this element:')
_N_e = _N.subs(physical_coords_map)
sp.pprint(_N_e)

print('\nThe shape function matrix N for this element at node 1:')
_N_e_node1 = _N_e.subs({'R': physical_coords['P1'][0], 'Z': physical_coords['P1'][2]})
sp.pprint(_N_e_node1.applyfunc(lambda x: x.round(4)))

# summing each row of the matrix
for i in range(ND):
    print(f'Sum of row {i + 1} of the shape function matrix N: {sum(_N_e_node1[i, :].applyfunc(lambda x: x.round(4)))}')


The shape function matrix N is (3, 12):

The shape function matrix N for this element:
⎡(0.25 - 0.25⋅R)⋅(1 - Z)             0                        0             (1 ↪
⎢                                                                              ↪
⎢           0             (0.25 - 0.25⋅R)⋅(1 - Z)             0                ↪
⎢                                                                              ↪
⎣           0                        0             (0.25 - 0.25⋅R)⋅(1 - Z)     ↪

↪  - Z)⋅(0.25⋅R + 0.25)             0                        0             (0. ↪
↪                                                                              ↪
↪          0             (1 - Z)⋅(0.25⋅R + 0.25)             0                 ↪
↪                                                                              ↪
↪          0                        0             (1 - Z)⋅(0.25⋅R + 0.25)      ↪

↪ 25⋅R + 0.25)⋅(Z + 1)             0                        0             (0.2 ↪
↪                   

### Jacobian matrix

In [6]:
def jacobian() -> sp.Matrix:
    """
    Calculate the jacobian matrix for the isoparametric mapping.
    """
    # The isoparametric mapping functions r(R,Z) and z(R,Z)
    mapping = isoparametric_mapping()
    r_map = mapping[0]
    z_map = mapping[1]

    # Calculate the derivatives
    dr_dR = sp.diff(r_map, R)
    dr_dZ = sp.diff(r_map, Z)
    dz_dR = sp.diff(z_map, R)
    dz_dZ = sp.diff(z_map, Z)

    return sp.Matrix([[dr_dR, dr_dZ],
                      [dz_dR, dz_dZ]])


# Calculate the jacobian matrix symbolically
J = jacobian()

# Substitute the symbolic coordinates with numerical values in the Jacobian
J_element = J.subs(physical_coords_map)

print('The jacobian matrix with coordinate values substituted is:')
sp.pprint(J_element)
print()

# Evaluate the jacobian matrix at some points
for r_eval in [-1, 1]:
    for z_eval in [-1, 1]:
        J_evaluated = J_element.subs({R: r_eval, Z: z_eval})
        print(f'Jacobian matrix at r={r_eval}, z={z_eval}:')
        sp.pprint(J_evaluated)
        print(f'The determinant at this point is |J|={J_evaluated.det():.2f}')
        print()

The jacobian matrix with coordinate values substituted is:
⎡10.0    0  ⎤
⎢           ⎥
⎣ 0    500.0⎦

Jacobian matrix at r=-1, z=-1:
⎡10.0    0  ⎤
⎢           ⎥
⎣ 0    500.0⎦
The determinant at this point is |J|=5000.00

Jacobian matrix at r=-1, z=1:
⎡10.0    0  ⎤
⎢           ⎥
⎣ 0    500.0⎦
The determinant at this point is |J|=5000.00

Jacobian matrix at r=1, z=-1:
⎡10.0    0  ⎤
⎢           ⎥
⎣ 0    500.0⎦
The determinant at this point is |J|=5000.00

Jacobian matrix at r=1, z=1:
⎡10.0    0  ⎤
⎢           ⎥
⎣ 0    500.0⎦
The determinant at this point is |J|=5000.00



# Strain vectors for $n=0$ and $n>0$ harmonics

### Recap: Strain Vector in the Axisymmetric Case 🔄

Here is the strain vector for the simple axisymmetric case ($n=0$) with $n$ being the number of the harmonic used.

The key simplification here is that displacements do not depend on the circumferential angle $\theta$, which means all partial derivatives with respect to $\theta$ are zero ($\frac{\partial}{\partial \theta} = 0$).

We start with the general strain-displacement equations and apply the axisymmetric simplification.

The displacement field is:
$$u_r = u_r(r, z)$$$$u_\theta = u_\theta(r, z)$$$$u_z = u_z(r, z)$$
Applying the condition $\frac{\partial}{\partial \theta} = 0$ to the general strain equations gives us the following components:

* **Radial Strain ($\epsilon_r$):**
    $$\epsilon_r = \frac{\partial u_r}{\partial r}$$

* **Hoop Strain ($\epsilon_\theta$):**
    $$\epsilon_\theta = \frac{1}{r}\underbrace{\frac{\partial u_\theta}{\partial \theta}}_{0} + \frac{u_r}{r} = \frac{u_r}{r}$$

* **Axial Strain ($\epsilon_z$):**
    $$\epsilon_z = \frac{\partial u_z}{\partial z}$$

* **Torsional Shear ($\gamma_{r\theta}$):**
    $$\gamma_{r\theta} = \frac{1}{r}\underbrace{\frac{\partial u_r}{\partial \theta}}_{0} + \frac{\partial u_\theta}{\partial r} - \frac{u_\theta}{r} = \frac{\partial u_\theta}{\partial r} - \frac{u_\theta}{r}$$

* **Torsional Shear ($\gamma_{\theta z}$):**
    $$\gamma_{\theta z} = \frac{\partial u_\theta}{\partial z} + \frac{1}{r}\underbrace{\frac{\partial u_z}{\partial \theta}}_{0} = \frac{\partial u_\theta}{\partial z}$$

* **In-Plane Shear ($\gamma_{zr}$):**
    $$\gamma_{zr} = \frac{\partial u_z}{\partial r} + \frac{\partial u_r}{\partial z}$$

This conveniently decouples the problem. The final strain vector has two independent sets of components. The first is the classical plane stress situation, where no torsion is present, and the second is the torsional problem, where only the circumferential displacement $u_\theta$ is involved.

1.  **An in-plane axial/radial problem** involving only $u_r$ and $u_z$:
    $$\vec{\epsilon}_{\text{plane}} = \begin{Bmatrix} \epsilon_r \\ \epsilon_\theta \\ \epsilon_z \\ \gamma_{zr} \end{Bmatrix} = \begin{Bmatrix} \frac{\partial u_r}{\partial r} \\ \frac{u_r}{r} \\ \frac{\partial u_z}{\partial z} \\ \frac{\partial u_z}{\partial r} + \frac{\partial u_r}{\partial z} \end{Bmatrix}$$

2.  A **torsional problem** involving only the circumferential displacement $u_\theta$:
    $$\vec{\epsilon}_{\text{torsion}} = \begin{Bmatrix} \gamma_{r\theta} \\ \gamma_{\theta z} \end{Bmatrix} = \begin{Bmatrix} \frac{\partial u_\theta}{\partial r} - \frac{u_\theta}{r} \\ \frac{\partial u_\theta}{\partial z} \end{Bmatrix}$$

For harmonics where $n>0$, the situation changes because the derivatives with respect to the angle $\theta$ are no longer zero. This introduces a crucial coupling that is absent in the axisymmetric ($n=0$) case.

***

### In-Plane Strains for a General Harmonic ($n>0$)

Let's look at the strain components analogous to the plane-stress state for a general harmonic $n$. For simplicity, we'll use the displacement field for a load symmetric about the $\theta=0$ plane:
$$u_r = u_{rn}(r, z) \cos(n\theta)$$$$u_\theta = u_{\theta n}(r, z) \sin(n\theta)$$$$u_z = u_{zn}(r, z) \cos(n\theta)$$
Here is how the four strain components are derived:

* **Radial Strain ($\epsilon_r$):** This calculation is straightforward.
    $$\epsilon_r = \frac{\partial u_r}{\partial r} = \frac{\partial u_{rn}}{\partial r} \cos(n\theta)$$

* **Hoop Strain ($\epsilon_\theta$):** This is the most significant change from the $n=0$ case.
    $$\epsilon_\theta = \frac{1}{r}\frac{\partial u_\theta}{\partial \theta} + \frac{u_r}{r}$$
    The derivative of $u_\theta$ with respect to $\theta$ is now non-zero:
    $$\epsilon_\theta = \frac{1}{r}\frac{\partial}{\partial \theta}(u_{\theta n} \sin(n\theta)) + \frac{u_{rn}\cos(n\theta)}{r} = \frac{1}{r}(n u_{\theta n} \cos(n\theta)) + \frac{u_{rn}\cos(n\theta)}{r}$$   $$\epsilon_\theta = \left(\frac{n u_{\theta n} + u_{rn}}{r}\right) \cos(n\theta)$$
    Notice that the hoop strain for $n>0$ now depends on **both the radial ($u_{rn}$) and tangential ($u_{\theta n}$) displacement amplitudes**. This is a fundamental difference from the $n=0$ case where it only depended on $u_r$.

* **Axial Strain ($\epsilon_z$):** This is also straightforward.
    $$\epsilon_z = \frac{\partial u_z}{\partial z} = \frac{\partial u_{zn}}{\partial z} \cos(n\theta)$$

* **In-Plane Shear ($\gamma_{zr}$):**
    $$\gamma_{zr} = \frac{\partial u_z}{\partial r} + \frac{\partial u_r}{\partial z} = \left(\frac{\partial u_{zn}}{\partial r} + \frac{\partial u_{rn}}{\partial z}\right) \cos(n\theta)$$

So, the vector of in-plane strains for any harmonic $n>0$ looks like this:
$$\vec{\epsilon}_{\text{plane}, n} = \begin{Bmatrix} \epsilon_r \\ \epsilon_\theta \\ \epsilon_z \\ \gamma_{zr} \end{Bmatrix}_n = \begin{Bmatrix} \frac{\partial u_{rn}}{\partial r} \\ \frac{n u_{\theta n} + u_{rn}}{r} \\ \frac{\partial u_{zn}}{\partial z} \\ \frac{\partial u_{zn}}{\partial r} + \frac{\partial u_{rn}}{\partial z} \end{Bmatrix} \cos(n\theta)$$


### Torsional/Shear Strains for a General Harmonic ($n>0$)

In the axisymmetric case ($n=0$), the torsional strains only depended on the tangential displacement $u_\theta$. For any harmonic where $n>0$, this is no longer true. The radial and axial displacements now also generate shear strains due to the twisting nature of the deformation. This is a key difference.

Here are the two shear strain components that make up what you called the "torsional part" of the strain vector, $\vec{\epsilon}_{\text{torsion},n}$, for a general harmonic $n$.

* **In-Plane Shear ($\gamma_{r\theta}$):**
    This component represents the shear distortion in the $r-\theta$ plane (the cross-sectional plane).
    $$\gamma_{r\theta} = \left(\frac{\partial u_{\theta n}}{\partial r} - \frac{u_{\theta n} + n u_{rn}}{r}\right) \sin(n\theta)$$
    **Explanation:**
    * The first part, $(\frac{\partial u_{\theta n}}{\partial r} - \frac{u_{\theta n}}{r})$, is the familiar term from the $n=0$ case, representing shear from pure torsion.
    * The new term, $-\frac{n u_{rn}}{r}$, is crucial. It shows that for $n>0$, the **radial displacement ($u_{rn}$) now creates in-plane shear**. As the cross-section deforms radially in a wavy pattern, it forces the material to shear.

***

* **Transverse Shear ($\gamma_{\theta z}$):**
    This component represents the shear distortion in the $\theta-z$ plane.
    $$\gamma_{\theta z} = \left(\frac{\partial u_{\theta n}}{\partial z} - \frac{n u_{zn}}{r}\right) \sin(n\theta)$$
    **Explanation:**
    * The term $\frac{\partial u_{\theta n}}{\partial z}$ is again the familiar part from the pure torsion case at $n=0$.
    * The new term, $-\frac{n u_{zn}}{r}$, shows that the **axial displacement ($u_{zn}$) now creates transverse shear**. As the structure bends, the axial fibers stretch and compress in a wavy $\cos(n\theta)$ pattern. This differential axial movement between adjacent circumferential points induces a shearing action. This is the primary shear effect in beam bending.

In summary, for any harmonic $n>0$, the problem becomes fully coupled. Unlike the simple axisymmetric case, the radial and axial displacements now directly contribute to the shear strains, linking bending and twisting phenomena together.

***

The matrix $[B_A]$ is a **6 x (3 * m)** matrix, where `m` is the number of nodes in the element. It's composed of `m` blocks, one for each node, with each block being a 6x3 matrix.

Let's look at the computation for a single node `i`. The block $[B_{Ai}]$ is the result of multiplying the 6x3 operator $[L_A]$ by the 3x3 shape function block for node `i`, $[N_i]$:
$$[B_{Ai}] = [L_A] [N_i]$$
Where:$$[L_A] = \begin{bmatrix} \frac{\partial}{\partial r} & 0 & 0 \\ \frac{1}{r} & 0 & 0 \\ 0 & 0 & \frac{\partial}{\partial z} \\ 0 & \frac{\partial}{\partial r} - \frac{1}{r} & 0 \\ 0 & \frac{\partial}{\partial z} & 0 \\ \frac{\partial}{\partial z} & 0 & \frac{\partial}{\partial r} \end{bmatrix} \quad \text{and} \quad [N_i] = \begin{bmatrix} N_i & 0 & 0 \\ 0 & N_i & 0 \\ 0 & 0 & N_i \end{bmatrix}$$

Performing this multiplication gives the correct **6x3 matrix block** for node `i`:

$$[B_{Ai}] = \begin{bmatrix} \frac{\partial N_i}{\partial r} & 0 & 0 \\ \frac{N_i}{r} & 0 & 0 \\ 0 & 0 & \frac{\partial N_i}{\partial z} \\ 0 & \frac{\partial N_i}{\partial r} - \frac{N_i}{r} & 0 \\ 0 & \frac{\partial N_i}{\partial z} & 0 \\ \frac{\partial N_i}{\partial z} & 0 & \frac{\partial N_i}{\partial r} \end{bmatrix}$$

That operator corresponds to the following standard strain order:

1.  **Radial Strain:** $\epsilon_r$
2.  **Hoop Strain:** $\epsilon_\theta$
3.  **Axial Strain:** $\epsilon_z$
4.  **In-Plane Shear:** $\gamma_{r\theta}$
5.  **Transverse Shear:** $\gamma_{\theta z}$
6.  **Transverse Shear:** $\gamma_{zr}$

The full $[B_A]$ matrix for the element is then formed by placing these blocks side-by-side:
$$[B_A] = \begin{bmatrix} [B_{A1}] & [B_{A2}] & \cdots & [B_{Am}] \end{bmatrix}$$


### The $\begin{bmatrix}B_A\end{bmatrix}$ matrix

In [7]:
def calculate_BA():
    """
    Calculates the complete BA-matrix for the element.
    # BA relates the strain vector [ε_r, ε_θ, ε_z, γ_θr, γ_θz, γ_zr]
    """
    B = sp.zeros(6, NNODE * ND)  # B matrix with 4 rows and NNODE * ND columns

    det_J = J_element.det()  # determinant of the jacobian matrix
    dNdr, dNdz = shape_function_derivatives()  # dN(i)/dr, dN(i)/dz

    # precalculating the derivatives of the isoparametric mapping
    dzdZ = sp.diff(y_map, Z).subs(physical_coords_map)
    drdZ = sp.diff(x_map, Z).subs(physical_coords_map)
    dzdR = sp.diff(y_map, R).subs(physical_coords_map)
    drdR = sp.diff(x_map, R).subs(physical_coords_map)

    r_iso = isoparametric_mapping()[0].subs(physical_coords_map)
    N_funcs = shape_functions().subs(physical_coords_map)

    for i in range(NNODE):

        # ∂Nᵢ/∂r = (1/|J|) * [ (∂z/∂Z * ∂Nᵢ/∂R) - (∂r/∂Z * ∂Nᵢ/∂Z) ]
        dNi_dr = (1 / det_J) * (dzdZ * dNdr[i] - drdZ * dNdz[i])

        # ∂Nᵢ/∂z = (1/|J|) * [ (∂r/∂R * ∂Nᵢ/∂Z) - (∂z/∂R * ∂Nᵢ/∂R) ]
        dNi_dz = (1 / det_J) * (drdR * dNdz[i] - dzdR * dNdr[i])

        Ni_r = N_funcs[i] / r_iso

        B[0, ND * i] = dNi_dr
        B[1, ND * i] = Ni_r
        B[2, ND * i + 2] = dNi_dz
        B[3, ND * i + 1] = dNi_dr - Ni_r
        B[4, ND * i + 1] = dNi_dz
        B[5, ND * i] = dNi_dz
        B[5, ND * i + 2] = dNi_dr

    return B

BA_element = calculate_BA()

sta = time.time()
_BA_evaluated = BA_element.subs({R: -0.5774, Z: -0.5774})
print(f'Time taken to calculate BAe matrix: {time.time() - sta:.4f} seconds')

# print only the first 3 columns of the matrix
print("\nThe first block of the BA matrix:")
_BA_evaluated_first3 = _BA_evaluated[:, :ND]
sp.pprint(_BA_evaluated_first3.applyfunc(lambda x: round(x, 8)))


Time taken to calculate BAe matrix: 0.0190 seconds

The first block of the BA matrix:
⎡-0.039435        0           0     ⎤
⎢                                   ⎥
⎢0.00061943       0           0     ⎥
⎢                                   ⎥
⎢    0            0       -0.0007887⎥
⎢                                   ⎥
⎢    0       -0.04005443      0     ⎥
⎢                                   ⎥
⎢    0       -0.0007887       0     ⎥
⎢                                   ⎥
⎣-0.0007887       0       -0.039435 ⎦


### The $\begin{bmatrix}B_B\end{bmatrix}$ matrix

In [8]:
def calculate_BB():
    """
    Calculates the complete BB-matrix for the element.
    BB relates the strain vector [ε_r, ε_θ, ε_z, γ_θr, γ_θz, γ_zr]
    """
    B = sp.zeros(6, NNODE * ND)  # B matrix with 4 rows and NNODE * ND columns

    r_iso = isoparametric_mapping()[0].subs(physical_coords_map)

    N_funcs = shape_functions().subs(physical_coords_map)

    for i in range(NNODE):

        Ni_r = N_funcs[i] / r_iso

        B[1, ND * i + 1] = Ni_r
        B[3, ND * i] = -Ni_r
        B[4, ND * i + 2] = -Ni_r

    return B

# Example of calculating and printing the B matrix
BB_element = calculate_BB()
# You can then substitute values as needed
BB_evaluated = BB_element.subs({R: -0.5774, Z: -0.5774})
# sp.pprint(BB_evaluated.evalf().applyfunc(lambda x: round(x, 3)))

# print only the first 3 columns of the matrix
print("\nThe first block of the BB matrix:")
BB_evaluated_first3 = BB_evaluated[:, :ND]
sp.pprint(BB_evaluated_first3.applyfunc(lambda x: round(x, 8)))


The first block of the BB matrix:
⎡     0           0            0     ⎤
⎢                                    ⎥
⎢     0       0.00061943       0     ⎥
⎢                                    ⎥
⎢     0           0            0     ⎥
⎢                                    ⎥
⎢-0.00061943      0            0     ⎥
⎢                                    ⎥
⎢     0           0       -0.00061943⎥
⎢                                    ⎥
⎣     0           0            0     ⎦


### The $\begin{bmatrix}B\end{bmatrix}$ matrix

$\begin{bmatrix}B\end{bmatrix} = \begin{bmatrix}B_A\end{bmatrix} + n\begin{bmatrix}B_B\end{bmatrix}$

In [9]:
def B_matrix(n: int) -> sp.Matrix:
    """
    Calculates the complete B-matrix for the element.
    B relates the strain vector [ε_r, ε_θ, ε_z, γ_θr, γ_θz, γ_zr]
    """
    BA = calculate_BA()
    BB = calculate_BB()

    return BA + n * BB

# Example of calculating and printing the B matrix
n = 0  # harmonic number
B_matrix_result = B_matrix(n)
# You can then substitute values as needed
B_evaluated_0 = B_matrix_result.subs({R: -0.5774, Z: -0.5774})
n = 1  # harmonic number
B_matrix_result = B_matrix(n)
# You can then substitute values as needed
B_evaluated_1 = B_matrix_result.subs({R: -0.5774, Z: -0.5774})
print("\nThe first block of the B matrices for n=0 and n=1:")

# Combine the two evaluated matrices for n=0 and n=1 for displaying
B_evaluated = sp.Matrix.hstack(B_evaluated_0[:, :ND], B_evaluated_1[:, :ND])
# B_evaluated = sp.Matrix((B_evaluated_0[:, :ND], B_evaluated_1[:, :ND]))

# B_evaluated_first3 = B_evaluated[:, :ND]
sp.pprint(B_evaluated.applyfunc(lambda x: round(x, 5)))


The first block of the B matrices for n=0 and n=1:
⎡-0.03944     0         0      -0.03944     0         0    ⎤
⎢                                                          ⎥
⎢0.00062      0         0      0.00062   0.00062      0    ⎥
⎢                                                          ⎥
⎢   0         0      -0.00079     0         0      -0.00079⎥
⎢                                                          ⎥
⎢   0      -0.04005     0      -0.00062  -0.04005     0    ⎥
⎢                                                          ⎥
⎢   0      -0.00079     0         0      -0.00079  -0.00062⎥
⎢                                                          ⎥
⎣-0.00079     0      -0.03944  -0.00079     0      -0.03944⎦


### The constitutive matrix $\begin{bmatrix}D\end{bmatrix}$

The constitutive matrix relates the stress vector to the strain vector. For isotropic material it is given by:
$$
\begin{bmatrix}D\end{bmatrix} = \frac{E}{(1 - 2\nu)(1 + \nu)} \begin{bmatrix}
1 - \nu & \nu & \nu & 0 & 0 & 0 \\
\nu & 1 - \nu & \nu & 0 & 0 & 0 \\
\nu & \nu & 1 - \nu & 0 & 0 & 0 \\
0 & 0 & 0 & \frac{1 - 2\nu}{2} & 0 & 0\\
0 & 0 & 0 & 0 & \frac{1 - 2\nu}{2} & 0\\
0 & 0 & 0 & 0 & 0 & \frac{1 - 2\nu}{2}
\end{bmatrix}
$$


In [10]:
def D_matrix():
    """
    The constitutive matrix for the general axissymmetric element.
    """
    D = sp.zeros(6, 6)
    D[0, 0] = 1 - nu
    D[0, 1] = nu
    D[0, 2] = nu

    D[1, 0] = nu
    D[1, 1] = 1 - nu
    D[1, 2] = nu

    D[2, 0] = nu
    D[2, 1] = nu
    D[2, 2] = 1 - nu

    D[3, 3] = D[4, 4] = D[5, 5] = (1 - 2 * nu) / 2

    D *= E / ((1 - 2 * nu) * (1 + nu))

    return D

D = D_matrix()
sp.pprint(D.applyfunc(lambda x: round(x / 1e9, 2)))

⎡269.23  115.38  115.38    0      0      0  ⎤
⎢                                           ⎥
⎢115.38  269.23  115.38    0      0      0  ⎥
⎢                                           ⎥
⎢115.38  115.38  269.23    0      0      0  ⎥
⎢                                           ⎥
⎢  0       0       0     76.92    0      0  ⎥
⎢                                           ⎥
⎢  0       0       0       0    76.92    0  ⎥
⎢                                           ⎥
⎣  0       0       0       0      0    76.92⎦


### Quadrature points and weights

In [11]:
if NNODE == 3:
    gi_data = {
        1: {
            'point': ((1/3, 1/3),),  # midpoint of the triangle
            'weight': (1,)
        },
        2: {
            # 'point': [-1 / np.sqrt(3), 1 / np.sqrt(3)],
            'point': [-0.5774, 0.5774],
            'weight': [1, 1]
        },
        3: {
            'point': ((1/6, 1/6), (4/6, 1/6), (1/6, 4/6), ),
            'weight': (1/3, 1/3, 1/3)
        },
    }
elif NNODE in (4, 8):
    gi_data = {
        1: {
            'point': [0],
            'weight': [2]
        },
        2: {
            # 'point': [-1 / np.sqrt(3), 1 / np.sqrt(3)],
            'point': [-0.5774, 0.5774],
            'weight': [1, 1]
        },
        3: {
            # 'point': [-np.sqrt(3 / 5), 0, np.sqrt(3 / 5)],
            'point': [-0.7745966, 0, 0.7745966],
            'weight': [5 / 9,  8 / 9, 5 / 9]
        },
        4: {
            'point': [-0.861136, -0.339981, 0.339981, 0.861136],
            'weight': [0.347855, 0.652145, 0.652145, 0.347855]
        },
        5: {
            'point': [-0.906180, -0.538469, 0, 0.906180, 0.538469],
            'weight': [0.236927, 0.478629, 0.568889, 0.478629, 0.236927]
        },
        6: {
            'point': [-0.932469, -0.661209, -0.238619, 0.238619, 0.661209, 0.932469],
            'weight': [0.171324, 0.360761, 0.467914, 0.467914, 0.360761, 0.171324]
        }
    }
else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

# The stiffness matrix for the element

The matrices $[K_0]$, $[K_1]$, and $[K_2]$ are the three fundamental, harmonic-independent stiffness matrices for a single element. They are calculated once and then used to instantly generate the stiffness matrix for any harmonic $n$.

Their purpose is to separate the geometry-dependent parts of the stiffness calculation from the harmonic-dependent parts.

***

# The Stiffness Matrix Formula

The element stiffness matrix for any harmonic $n$, denoted $[k_e]_n$, is constructed using this simple quadratic formula:
$$[k_e]_n = [K_0] + n [K_1] + n^2 [K_2]$$

***

# Origin of Each Matrix

These matrices arise from expanding the integral for the element stiffness matrix after separating the strain-displacement matrix $[B_n]$ into its harmonic-independent part $[B_A]$ and its linear-in-$n$ part $[B_B]$.

* **[K₀]: The Base Stiffness Matrix**
    $$[K_0] = \int_V [B_A]^T [D] [B_A] dV$$
    This matrix is derived from the parts of the strain calculation that **do not** have an explicit $n$ coefficient. It is essentially the stiffness matrix you would use for a standard axisymmetric analysis ($n=0$).

* **[K₁]: The Linear Coupling Matrix**
    $$[K_1] = \int_V ([B_A]^T [D] [B_B] + [B_B]^T [D] [B_A]) dV$$
    This matrix represents the **coupling** between the base strain terms and the harmonic-dependent strain terms. It is the part of the stiffness that grows linearly with the harmonic number $n$.

* **[K₂]: The Harmonic Stiffness Matrix**
    $$[K_2] = \int_V [B_B]^T [D] [B_B] dV$$
    This matrix is derived purely from the parts of the strain calculation that are **explicitly multiplied by $n$**. It represents the stiffness that arises solely from the non-axisymmetric deformation modes and grows with the square of the harmonic number, $n^2$.

In [12]:
def K0_matrix(ng) -> np.array:
    """
    Gets the K0 stiffness matrix for the element.
    """
    K_lok = sp.zeros(NNODE * ND, NNODE * ND)  # initialize the stiffness matrix
    # J = jacobian()
    # J_substituted = J.subs(physical_coords_map)
    D = D_matrix()
    r_iso = isoparametric_mapping()[0]  # r(R, Z) mapping

    if NNODE == 3:
        for p, w in zip(gi_data[ng]['point'], gi_data[ng]['weight']):
            r_gp, z_gp = p[0], p[1]  # Gauss point in natural coords

            # Evaluate B, det(J), and r at the Gauss point
            BA = BA_element.subs({R: r_gp, Z: z_gp}).evalf()
            J_evaluated = J_element.subs({R: r_gp, Z: z_gp}).evalf()
            det_J = J_evaluated.det()

            Area = det_J / 2  # Area of the triangle

            r_physical_at_gp = r_iso.subs(physical_coords_map).subs({R: r_gp, Z: z_gp}).evalf()

            # Calculate the stiffness matrix contribution with the 2*pi*r term
            K_contribution = 2 * sp.pi * Area * BA.T * D * BA * r_physical_at_gp * w
            K_lok += K_contribution

    elif NNODE in (4, 8):

        for i in range(ng):
            for j in range(ng):
                s = gi_data[ng]['point'][i]
                t = gi_data[ng]['point'][j]
                w = gi_data[ng]['weight'][i] * gi_data[ng]['weight'][j]

                # Calculate the jacobian at the gauss point
                J_evaluated = J_element.subs({R: s, Z: t}).evalf()
                det_J = J_evaluated.det()
                r_physical_at_gp = r_iso.subs(physical_coords_map).subs({R: s, Z: t}).evalf()
                BA = BA_element.subs({R: s, Z: t}).evalf()

                # Calculate the stiffness matrix contribution
                K_contribution = 2 * sp.pi * BA.T * D * BA * r_physical_at_gp * det_J * w
                K_lok += K_contribution

    else:
        raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

    return K_lok


ng = NGS[0]  # number of Gauss points, can be 1, 2, 3, 4, 5, or 6
K0 = K0_matrix(ng)
K0_1np = np.array(K0).astype(np.float64)

ng = NGS[1]
K0 = K0_matrix(ng)
K0_3np = np.array(K0).astype(np.float64)

print("The K0 stiffness matrix with {} gauss points:".format(NGS[0]))
print(np.round(K0_1np / 1e7, 4))
print()
print("The K0 stiffness matrix with {} gauss points:".format(NGS[1]))
print(np.round(K0_3np / 1e7, 4))


The K0 stiffness matrix with 1 gauss points:
[[ 2.11800772e+09  0.00000000e+00  3.03284522e+07 -2.13522541e+09
   0.00000000e+00  5.92069385e+06 -2.13571357e+09  0.00000000e+00
  -3.03284522e+07  2.11751956e+09  0.00000000e+00 -5.92069385e+06]
 [ 0.00000000e+00  6.22580901e+08  0.00000000e+00  0.00000000e+00
  -6.09890063e+08  0.00000000e+00  0.00000000e+00 -6.10378218e+08
   0.00000000e+00  0.00000000e+00  6.22092746e+08  0.00000000e+00]
 [ 3.03284522e+07  0.00000000e+00  6.11048229e+08 -6.28318531e+06
   0.00000000e+00 -6.09339686e+08 -3.06909436e+07  0.00000000e+00
  -6.11048229e+08  5.92069385e+06  0.00000000e+00  6.09339686e+08]
 [-2.13522541e+09  0.00000000e+00 -6.28318531e+06  2.15425686e+09
   0.00000000e+00 -3.06909436e+07  2.15376871e+09  0.00000000e+00
   6.28318531e+06 -2.13571357e+09  0.00000000e+00  3.06909436e+07]
 [ 0.00000000e+00 -6.09890063e+08  0.00000000e+00  0.00000000e+00
   5.98414804e+08  0.00000000e+00  0.00000000e+00  5.97926649e+08
   0.00000000e+00  0.000000

In [13]:
def K1_matrix(ng) -> np.array:
    """
    Gets the K1 stiffness matrix for the element.
    """
    K_lok = sp.zeros(NNODE * ND, NNODE * ND)  # initialize the stiffness matrix
    # J = jacobian()
    # J_substituted = J.subs(physical_coords_map)
    D = D_matrix()
    r_iso = isoparametric_mapping()[0].subs(physical_coords_map)  # r(R, Z) mapping

    if NNODE == 3:
        for p, w in zip(gi_data[ng]['point'], gi_data[ng]['weight']):
            r_gp, z_gp = p[0], p[1]  # Gauss point in natural coords

            # Evaluate B, det(J), and r at the Gauss point
            BA = BA_element.subs({R: r_gp, Z: z_gp}).evalf()
            BB = BB_element.subs({R: r_gp, Z: z_gp}).evalf()
            J_evaluated = J_element.subs({R: r_gp, Z: z_gp}).evalf()
            det_J = J_evaluated.det()
            r_physical_at_gp = r_iso.subs({R: r_gp, Z: z_gp}).evalf()

            # Calculate the stiffness matrix contribution with the 2*pi*r term
            K_contribution = 1 * sp.pi * (BA.T * D * BB + BB.T * D * BA) * r_physical_at_gp * det_J * w
            K_lok += K_contribution

    elif NNODE in (4, 8):

        for i in range(ng):
            for j in range(ng):
                s = gi_data[ng]['point'][i]
                t = gi_data[ng]['point'][j]
                w = gi_data[ng]['weight'][i] * gi_data[ng]['weight'][j]

                # Calculate the jacobian at the gauss point
                J_evaluated = J_element.subs({R: s, Z: t}).evalf()
                det_J = J_evaluated.det()
                r_physical_at_gp = r_iso.subs(physical_coords_map).subs({R: s, Z: t}).evalf()
                BA = BA_element.subs({R: s, Z: t}).evalf()
                BB = BB_element.subs({R: s, Z: t}).evalf()

                # Calculate the stiffness matrix contribution
                # K_contribution = 2 * sp.pi * BA.T * D * BA * r_physical_at_gp * det_J * w
                K_contribution = 1 * sp.pi * (BA.T * D * BB + BB.T * D * BA) * r_physical_at_gp * det_J * w
                K_lok += K_contribution

    else:
        raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

    return K_lok


# ng = NGS[0]  # number of Gauss points, can be 1, 2, 3, 4, 5, or 6
# K1 = K1_matrix(ng)
# K1_1np = np.array(K1).astype(np.float64)
#
# ng = NGS[1]
# K1 = K1_matrix(ng)
# K1_3np = np.array(K1).astype(np.float64)
#
# print(np.round(K1_1np / 1e7, 1))
# print()
# print(np.round(K1_3np / 1e7, 1))

In [14]:
def K2_matrix(ng) -> np.array:
    """
    Gets the K2 stiffness matrix for the element.
    """
    K_lok = sp.zeros(NNODE * ND, NNODE * ND)  # initialize the stiffness matrix
    # J = jacobian()
    # J_substituted = J.subs(physical_coords_map)
    D = D_matrix()
    r_iso = isoparametric_mapping()[0].subs(physical_coords_map)  # r(R, Z) mapping

    if NNODE == 3:
        for p, w in zip(gi_data[ng]['point'], gi_data[ng]['weight']):
            r_gp, z_gp = p[0], p[1]  # Gauss point in natural coords

            # Evaluate B, det(J), and r at the Gauss point
            BB = BB_element.subs({R: r_gp, Z: z_gp}).evalf()
            J_evaluated = J_element.subs({R: r_gp, Z: z_gp}).evalf()
            det_J = J_evaluated.det()
            r_physical_at_gp = r_iso.subs({R: r_gp, Z: z_gp}).evalf()

            # Calculate the stiffness matrix contribution with the 2*pi*r term
            K_contribution = sp.pi * BB.T * D * BB * r_physical_at_gp * det_J * w
            K_lok += K_contribution

    elif NNODE in (4, 8):

        for i in range(ng):
            for j in range(ng):
                s = gi_data[ng]['point'][i]
                t = gi_data[ng]['point'][j]
                w = gi_data[ng]['weight'][i] * gi_data[ng]['weight'][j]

                # Calculate the jacobian at the gauss point
                J_evaluated = J_element.subs({R: s, Z: t}).evalf()
                det_J = J_evaluated.det()
                r_physical_at_gp = r_iso.subs({R: s, Z: t}).evalf()
                BB = BB_element.subs({R: s, Z: t}).evalf()

                # Calculate the stiffness matrix contribution
                K_contribution = 2 * sp.pi * BB.T * D * BB * r_physical_at_gp * det_J * w
                K_lok += K_contribution

    else:
        raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

    return K_lok


ng = NGS[0]  # number of Gauss points, can be 1, 2, 3, 4, 5, or 6
K2 = K2_matrix(ng)
K2_1np = np.array(K2).astype(np.float64)

print()
ng = NGS[1]
K2 = K2_matrix(ng)
K2_3np = np.array(K2).astype(np.float64)

print(np.round(K2_1np / 1e7, 1))
print()
print(np.round(K2_3np / 1e7, 1))



[[ 59817.1      0.       0.   59817.1      0.       0.   59817.1      0.
       0.   59817.1      0.       0. ]
 [     0.  209359.8      0.       0.  209359.8      0.       0.  209359.8
       0.       0.  209359.8      0. ]
 [     0.       0.   59817.1      0.       0.   59817.1      0.       0.
   59817.1      0.       0.   59817.1]
 [ 59817.1      0.       0.   59817.1      0.       0.   59817.1      0.
       0.   59817.1      0.       0. ]
 [     0.  209359.8      0.       0.  209359.8      0.       0.  209359.8
       0.       0.  209359.8      0. ]
 [     0.       0.   59817.1      0.       0.   59817.1      0.       0.
   59817.1      0.       0.   59817.1]
 [ 59817.1      0.       0.   59817.1      0.       0.   59817.1      0.
       0.   59817.1      0.       0. ]
 [     0.  209359.8      0.       0.  209359.8      0.       0.  209359.8
       0.       0.  209359.8      0. ]
 [     0.       0.   59817.1      0.       0.   59817.1      0.       0.
   59817.1      0.       0.

### The stiffness matrix $\begin{bmatrix}K\end{bmatrix}$

In [15]:
def K_matrix(n: int, ng: int) -> np.array:
    """
    Gets the stiffness matrix for the element for a given harmonic n.
    """
    K0 = np.array(K0_matrix(ng)).astype(np.float64)
    K2 = np.array(K2_matrix(ng)).astype(np.float64)

    return K0 + ((n ** 2) * K2)
    K1 = np.array(K1_matrix(ng)).astype(np.float64)
    return K0 + n * K1 + n**2 * K2


# Loads

### The Load Vector $\begin{bmatrix}F\end{bmatrix}$

The next step is to formulate the **consistent nodal load vector** for each harmonic, $\{F_n\}$.

Just as we decomposed the displacement response into a Fourier series, the applied external load must also be decomposed in the same way.

***

### ## 1. Load Decomposition

You must first represent your specific applied load (e.g., wind pressure, a point force) as a Fourier series in the circumferential ($\theta$) direction. For a general surface pressure $\vec{p}$ with components $(p_r, p_\theta, p_z)$, this looks like:
$$p_r(r, \theta, z) = \sum_{n=0}^{\infty} p_{rn}(r, z) \cos(n\theta)$$$$p_\theta(r, \theta, z) = \sum_{n=0}^{\infty} p_{\theta n}(r, z) \sin(n\theta)$$$$p_z(r, \theta, z) = \sum_{n=0}^{\infty} p_{zn}(r, z) \cos(n\theta)$$
You would perform a Fourier analysis on your load to find the amplitude functions ($p_{rn}, p_{\theta n}, p_{zn}$). For many common loads, these are standard or simple to derive.

***

### ## 2. Nodal Force Vector Formulation

Once you have the load amplitudes for each harmonic $n$, you can calculate the corresponding consistent nodal force vector, $\{F_n\}$. This is done using the shape functions $[N]$ and the principle of virtual work. The formula for the element nodal force vector is:
$$\{F_n\}_e = \int_A [N]^T \{p_n\} dA$$
Where $\{p_n\}$ is the vector of load amplitudes $\{p_{rn}, p_{\theta n}, p_{zn}\}^T$ for that harmonic. Just like with the stiffness matrix, the integral is taken over the 2D domain, and the circumferential part is handled analytically.

***

### ## 3. Assembly and Solution ⚙️

With all the components ready, the final procedure is:
1.  **Choose Harmonics:** Decide which harmonics ($n=0, 1, 2, ...$) are significant based on your load decomposition.
2.  **For each harmonic $n$**:
    * Assemble the global stiffness matrix $[K_n]$ from the element matrices $[k_e]_n$.
    * Assemble the global force vector $\{F_n\}$ from the element force vectors $\{F_n\}_e$.
    * Solve the system of linear equations: $[K_n]\{u_n\} = \{F_n\}$ to find the nodal displacement amplitudes $\{u_n\}$.
3.  **Superimpose Results:** The final displacement is the sum of the solutions from all the calculated harmonics. The stresses are calculated similarly and also summed.

### Hydrostatic Pressure Load Calculation

In [16]:
def get_hydrostatic_pressure_load(
    r: float,
    z: float,
    inner_radius: float,
    fluid_level_H: float,
    fluid_gamma: float
) -> np.array:
    """
    Calculates the load vector (pr, p_theta, pz) for hydrostatic pressure.

    Args:
        r: The radial coordinate of the point.
        z: The vertical coordinate of the point.
        inner_radius: The inner radius of the cylinder.
        fluid_level_H: The vertical coordinate of the fluid's free surface.
        fluid_gamma: The specific weight of the fluid (e.g., density * g).
    """
    p_r = 0.0
    p_theta = 0.0
    p_z = 0.0

    # Check if the point is on the inner surface AND below the fluid level
    if np.isclose(r, inner_radius) and z < fluid_level_H:
        # Calculate the depth of the point below the fluid surface
        depth = fluid_level_H - z

        # Calculate the hydrostatic pressure
        pressure = fluid_gamma * depth

        p_r = pressure

    return np.array([p_r, p_theta, p_z]).reshape(-1, 1)

# # --- Example Usage ---
# # Using consistent units: MPa and mm
# # Specific weight of water (gamma = rho * g) in MPa/mm
# # gamma = 9.81e-6 MPa/mm
# WATER_GAMMA = 9.81e-6
# INNER_RADIUS = 1000.0 # mm
# FLUID_LEVEL = 1500.0  # mm (Fluid surface is at z = 1500)
#
# # 1. Point submerged 500 mm below the surface (at z=1000)
# load_submerged = get_hydrostatic_pressure_load(r=1000.0, z=1000.0,
#                                               inner_radius=INNER_RADIUS,
#                                               fluid_level_H=FLUID_LEVEL,
#                                               fluid_gamma=WATER_GAMMA)
# print(f"Load at z=1000mm: {load_submerged}")
# print(f"Pressure value: {WATER_GAMMA * (1500 - 1000):2g} MPa\n")
#
#
# # 2. Point at the bottom (at z=0)
# load_at_bottom = get_hydrostatic_pressure_load(r=1000.0, z=0.0,
#                                                inner_radius=INNER_RADIUS,
#                                                fluid_level_H=FLUID_LEVEL,
#                                                fluid_gamma=WATER_GAMMA)
# print(f"Load at z=0mm: {load_at_bottom}")
# print(f"Pressure value: {WATER_GAMMA * (1500 - 0):2g} MPa\n")
#
#
# # 3. Point above the fluid level (at z=2000)
# load_above_surface = get_hydrostatic_pressure_load(r=1000.0, z=2000.0,
#                                                    inner_radius=INNER_RADIUS,
#                                                    fluid_level_H=FLUID_LEVEL,
#                                                    fluid_gamma=WATER_GAMMA)
# print(f"Load at z=2000mm: {load_above_surface}")

### Assembling the Element Force Vector

In [17]:
def get_hydrostatic_load_force_vector():

    # Initialize the element force vector
    F_element = np.zeros((ND * NNODE, 1))

    if NNODE == 3:
        # loaded nodes are: 1, 3

        # for this the distance between nodes P1 and P4
        pts = physical_coords
        L_edge = np.sqrt((pts['P1'][0] - pts['P3'][0])**2 + (pts['P1'][2] - pts['P3'][2])**2)
        print('L_edge', L_edge)
        det_J_edge = L_edge / 2.0
        print('det_J_edge', det_J_edge)

        # Use 1 Gauss points for the line integral
        gauss_points = gi_data[2]['point']
        gauss_weights = gi_data[2]['weight']

        for i in range(len(gauss_points)):
            s_gp = gauss_points[i]
            w_gp = gauss_weights[i]

            # Evaluate 1D shape functions at the Gauss point s_gp
            N3_at_gp = 0.5 * (1 - s_gp)
            N1_at_gp = 0.5 * (1 + s_gp)

            # 2. Find the physical coordinates (r, z) of the Gauss point
            #    by interpolating the nodal coordinates (r1, z1) and (r4, z4)
            r_gp = float((N1_at_gp * r1 + N3_at_gp * r3).subs(physical_coords_map).evalf())
            z_gp = float((N1_at_gp * z1 + N3_at_gp * z3).subs(physical_coords_map).evalf())

            # 3. Calculate the pressure vector at this Gauss point
            # p_vector is a (3, 1) vector: [p_r, p_theta, p_z]
            p_vector = get_hydrostatic_pressure_load(
                r=float(r_gp),
                z=float(z_gp),
                inner_radius=float(r_gp),
                fluid_level_H=100000.0,  # Fluid level height
                fluid_gamma=1e-5  # 9.81e-6,     # Specific weight of water in N/mm3
            )

            # Construct the 3x9 [N] matrix for this point on the edge
            # Only columns for nodes 1 and 3 will be non-zero
            N_matrix_at_gp = np.zeros((3, 9))
            # Node P3 contributions
            N_matrix_at_gp[0, 0] = N3_at_gp
            N_matrix_at_gp[1, 1] = N3_at_gp
            N_matrix_at_gp[2, 2] = N3_at_gp
            # Node P1 contributions
            N_matrix_at_gp[0, 6] = N1_at_gp
            N_matrix_at_gp[1, 7] = N1_at_gp
            N_matrix_at_gp[2, 8] = N1_at_gp

            sp.pprint(N_matrix_at_gp)

            # Calculate the contribution at this Gauss point
            F_contribution = 2 * np.pi * r_gp * (N_matrix_at_gp.T @ p_vector) * det_J_edge * w_gp

            # Add to the total element force vector
            F_element += F_contribution

        # F_element now holds the consistent nodal forces for the element.
        return F_element

    elif NNODE == 4:

        # loaded nodes are: 1, 4

        # for this the distance between nodes P1 and P4
        pts = physical_coords
        L_edge = np.sqrt((pts['P1'][0] - pts['P4'][0])**2 + (pts['P1'][2] - pts['P4'][2])**2)
        det_J_edge = L_edge / 2.0

        # Use 2 Gauss points for the line integral
        gauss_points = gi_data[2]['point']
        gauss_weights = gi_data[2]['weight']

        for i in range(len(gauss_points)):
            s_gp = gauss_points[i]
            w_gp = gauss_weights[i]

            # Evaluate 1D shape functions at the Gauss point s_gp
            N4_at_gp = 0.5 * (1 - s_gp)
            N1_at_gp = 0.5 * (1 + s_gp)

            # 2. Find the physical coordinates (r, z) of the Gauss point
            #    by interpolating the nodal coordinates (r1, z1) and (r4, z4)
            r_gp = float((N1_at_gp * r1 + N4_at_gp * r4).subs(physical_coords_map).evalf())
            z_gp = float((N1_at_gp * z1 + N4_at_gp * z4).subs(physical_coords_map).evalf())

            # 3. Calculate the pressure vector at this Gauss point
            # p_vector is a (3, 1) vector: [p_r, p_theta, p_z]
            p_vector = get_hydrostatic_pressure_load(
                r=float(r_gp),
                z=float(z_gp),
                inner_radius=float(r_gp),
                fluid_level_H=100000.0,  # Fluid level height
                fluid_gamma=1e-5  # 9.81e-6,     # Specific weight of water in N/mm3
            )

            print('p_vector.T:', p_vector.T)


            # Construct the 3x8 [N] matrix for this point on the edge
            # Only columns for nodes 1 and 2 will be non-zero
            N_matrix_at_gp = np.zeros((3, 12))
            # Node P1 contributions
            N_matrix_at_gp[0, 0] = N4_at_gp
            N_matrix_at_gp[1, 1] = N4_at_gp
            N_matrix_at_gp[2, 2] = N4_at_gp
            # Node P4 contributions
            N_matrix_at_gp[0, 9] = N1_at_gp
            N_matrix_at_gp[1, 10] = N1_at_gp
            N_matrix_at_gp[2, 11] = N1_at_gp

            # Calculate the contribution at this Gauss point
            F_contribution = 2 * np.pi * r_gp * (N_matrix_at_gp.T @ p_vector) * det_J_edge * w_gp

            # Add to the total element force vector
            F_element += F_contribution

        # F_element now holds the consistent nodal forces for the element.
        return F_element

    elif NNODE == 8:

        # Knoten auf der Kante: 1, 8, 4
        pts = physical_coords
        # Kantenlänge und Jacobi-Determinante
        L_edge = np.sqrt((pts['P1'][0] - pts['P4'][0])**2 + (pts['P1'][2] - pts['P4'][2])**2)
        det_J_edge = L_edge / 2.0

        # 3-Punkt-Gauß-Integration für die Kante
        gauss_points = [-np.sqrt(3/5), 0.0, np.sqrt(3/5)]
        gauss_weights = [5/9, 8/9, 5/9]

        # 1D-Shape-Funktionen für 3 Knoten
        def N1(s): return 0.5 * s * (s - 1)
        def N8(s): return (1 - s**2)
        def N4(s): return 0.5 * s * (s + 1)

        for s_gp, w_gp in zip(gauss_points, gauss_weights):
            # Interpolierte Koordinaten am Gauß-Punkt
            r_gp = float((N1(s_gp) * r1 + N8(s_gp) * r5 + N4(s_gp) * r4).subs(physical_coords_map))
            z_gp = float((N1(s_gp) * z1 + N8(s_gp) * z5 + N4(s_gp) * z4).subs(physical_coords_map))

            # Druckvektor am Gauß-Punkt
            p_vector = get_hydrostatic_pressure_load(
                r=r_gp,
                z=z_gp,
                inner_radius=r_gp,
                fluid_level_H=100000.0,
                fluid_gamma=1e-5
            )

            print('p_vector.T:', p_vector.T)

            # Shape-Funktionen am Gauß-Punkt
            N_vec = np.zeros((NNODE, 1))
            N_vec[0] = N1(s_gp)
            N_vec[7] = N8(s_gp)
            N_vec[3] = N4(s_gp)

            # Beitrag zum Kraftvektor
            for i in range(NNODE):
                for d in range(ND):
                    F_element[ND * i + d, 0] += N_vec[i, 0] * p_vector[d, 0] * 2 * np.pi * r_gp * det_J_edge * w_gp

        # F_element now holds the consistent nodal forces for the element.
        return F_element

    else:
        raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

print("Element Force Vector (F_element):")
sp.pprint(get_hydrostatic_load_force_vector())
print(sum(get_hydrostatic_load_force_vector()))

Element Force Vector (F_element):
p_vector.T: [[0.992113 0.       0.      ]]
p_vector.T: [[0.997887 0.       0.      ]]
[[3120647.80050989] 
 [      0.        ] 
 [      0.        ] 
 [      0.        ] 
 [      0.        ] 
 [      0.        ] 
 [      0.        ] 
 [      0.        ] 
 [      0.        ] 
 [3131121.5801338 ] 
 [      0.        ] 
 [      0.        ]]
p_vector.T: [[0.992113 0.       0.      ]]
p_vector.T: [[0.997887 0.       0.      ]]
[6251769.38064369]


# Solving the System of Equations

In [18]:
# Applying the boundary conditions.
def apply_boundary_conditions(K, F, fixed_dofs):
    """
    Applies boundary conditions to the stiffness matrix K and force vector F.
    Fixed nodes are removed from the system.
    """
    # Remove rows and columns corresponding to fixed nodes
    for dof in fixed_dofs:
        # setting the node row to zeros
        K[dof, :] = 0
        K[:, dof] = 0
        K[dof, dof] = 1  # Set diagonal to 1 for fixed nodes
        F[dof] = 0

    return K, F

# Solve the system of equations
def solve_system(K, F):
    """
    Solves the system of equations K * u = F for u.
    """
    # Convert to numpy arrays for numerical solving
    K_np = np.array(K).astype(np.float64)
    F_np = np.array(F).astype(np.float64)

    # Solve the linear system
    u = np.linalg.solve(K_np, F_np)

    return u

def get_reactionforces(K, F, u):
    """
    Calculates the reaction forces at the fixed nodes.
    """
    # Calculate the reaction forces
    reaction_forces = K @ u - F

    return reaction_forces

K = K_matrix(0, NGS[1])
F = get_hydrostatic_load_force_vector()


if NNODE == 3:
    # fixed_dofs=[2, 5, 8, ]  # Fixed nodes for 3-node element
    fixed_dofs=[2, 5, ]  # open end
elif NNODE == 4:
    fixed_dofs=[2, 5, 8, 11]
    fixed_dofs=[2, 5]  # open end
elif NNODE == 8:
    fixed_dofs=[2, 5, 8, 11, 14, 20, 17, 23]  # Fixed nodes for 8-node element
    # fixed_dofs=[2, 5]  # open end
else:
    raise ValueError("Unsupported number of nodes. Only 4 or 8 nodes are supported")

K, F = apply_boundary_conditions(K, F, fixed_dofs=fixed_dofs)
u = solve_system(K, F)
# sp.pprint(u)
# print("\n")



p_vector.T: [[0.992113 0.       0.      ]]
p_vector.T: [[0.997887 0.       0.      ]]


# Getting the stress vector

In [19]:
def get_stress_vector(u, n):

    _D = D_matrix()

    if NNODE == 3:
        # raise ValueError("This example is for a 4-noded quad element. Please use a 4-noded element.")

        Be = B_matrix(n).subs(physical_coords_map)

        for i in range(NGS[0]):
            for j in range(NGS[0]):
                s = gi_data[ng]['point'][i][0]
                t = gi_data[ng]['point'][j][0]

                B = Be.subs({R: s, Z: t}).evalf()

                epsilon_vector = B @ u  # Result is a (6, 1) vector

                # print()
                # print(f"Gauss point (s, t): {s}, {t}")
                # print('epsilon:', epsilon_vector)

                stress_vector = _D @ epsilon_vector # Result is a (6, 1) vector

                # print('sigma:', stress_vector)

    elif NNODE in (4, 8):

        Be = B_matrix(n).subs(physical_coords_map)

        for i in range(NGS[1]):
            for j in range(NGS[1]):
                s = gi_data[ng]['point'][i]
                t = gi_data[ng]['point'][j]

                B = Be.subs({R: s, Z: t}).evalf()

                epsilon_vector = B @ u  # Result is a (6, 1) vector

                # print()
                # print(f"Gauss point (s, t): {s}, {t}")
                # print('epsilon:', epsilon_vector)

                stress_vector = _D @ epsilon_vector # Result is a (6, 1) vector

                # print('sigma:', stress_vector)

        # # F_element now holds the consistent nodal forces for the element.
        # return F_element

    else:
        raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

    return stress_vector
# print("Stresses are:")
# sp.pprint(get_stress_vector(u))

### A harmonic loading: bending moment

In [20]:
def apply_bending_moment(M: float):

    # Apply a bending moment at node 1
    bending_moment = M  # kNm
    bending_moment *= 1e6  # Convert to Nmm for consistency with other units

    # The bending moment is applied as a force at the node
    # For an 8-node element, we apply it to the first node (node 1)
    F_element = np.zeros((ND * NNODE, 1))
    _R = (physical_coords['P3'][0] + physical_coords['P4'][0]) / 2  # Average radius of the element
    _F = -bending_moment / _R  # Convert to force per unit length

    # print("Bending moment (M):", bending_moment)
    # print("Physical radius at node 1 (r1):", _R)
    # print("Bending moment (F_element):", _F)


    if NNODE == 3:
        raise ValueError('Not yet implemented for 3-node elements')

    elif NNODE == 4:
        # the force is applied in the z-direction over the nodes of the top edge.
        # the affected nodes are (n): 3, 4, 7
        # the affected DOF indices are (3n-1): 8, 11, 20
        F_element[8, 0] = _F / 2  # Node 3
        F_element[11, 0] = _F / 2  # Node 4


    else:

        # the force is applied in the z-direction over the nodes of the top edge.
        # the affected nodes are (n): 3, 4, 7
        # the affected DOF indices are (3n-1): 8, 11, 20
        F_element[8, 0] = _F / 6  # Node 3
        F_element[20, 0] = 4 * _F / 6  # Node 7
        F_element[11, 0] = _F / 6  # Node 4

    return F_element

def apply_axial_force(N: float):

    # Apply a bending moment at node 1
    axial_force = N  # kN
    axial_force *= 1e3  # Convert to Nmm for consistency with other units
    _F = axial_force  # Convert to force per unit length

    _R = physical_coords['P2'][0]
    _r = physical_coords['P1'][0]
    _A = (_R ** 2 - _r ** 2) * np.pi  # Cross-sectional area of the cylinder

    # The bending moment is applied as a force at the node
    # For an 8-node element, we apply it to the first node (node 1)
    F_element = np.zeros((ND * NNODE, 1))

    if NNODE == 3:
        raise ValueError('Not yet implemented for 3-node elements')

    elif NNODE == 4:
        # the force is applied in the z-direction over the nodes of the top edge.
        # the affected nodes are (n): 3, 4, 7
        # the affected DOF indices are (3n-1): 8, 11, 20
        F_element[8, 0] = _F / 2  # Node 3
        F_element[11, 0] = _F / 2  # Node 4


    else:

        # the force is applied in the z-direction over the nodes of the top edge.
        # the affected nodes are (n): 3, 4, 7
        # the affected DOF indices are (3n-1): 8, 11, 20
        F_element[8, 0] = _F / 6  # Node 3
        F_element[20, 0] = 4 * _F / 6  # Node 7
        F_element[11, 0] = _F / 6  # Node 4

    return F_element

def apply_unit_bending_rotation():
    """returns a displacement vector"""
    u_vector = np.zeros((ND * NNODE, 1))
    if NNODE == 3:
        raise ValueError('Not yet implemented for 3-node elements')

    elif NNODE == 4:
        # Apply a unit rotation at node 1
        u_vector[8, 0] = -0.01  # Node 3
        u_vector[11, 0] = 0.01  # Node 4

    else:
        # Apply a unit rotation at node 1
        u_vector[8, 0] = -0.01  # Node 3
        u_vector[11, 0] = 0.01  # Node 4

    return u_vector


# print("Bending moment load vector (F_element):")
# sp.pprint(apply_bending_moment())

In [21]:
if NNODE == 3:
    # fixed_dofs=[2, 5, 8, ]  # Fixed nodes for 3-node element
    fixed_dofs=[2, 5, ]  # open end
elif NNODE == 4:
    fixed_dofs=[2, 5]  # open end
elif NNODE == 8:
    fixed_dofs=[2, 5, 14]  # open end
else:
    raise ValueError("Unsupported number of nodes. Only 4 or 8 nodes are supported")

K_full = K_matrix(n=1, ng=NGS[1])
F_full = apply_bending_moment(M=1000)
# F_full = apply_axial_force(N=1000)
K, F = apply_boundary_conditions(K_full, F_full, fixed_dofs=fixed_dofs)
u = solve_system(K, F)
print('Case: bending moment')
print("Stresses are:")
sp.pprint(get_stress_vector(u, n=1)[2, 0])
sp.pprint(2*get_stress_vector(u, n=1)[2, 0])

print()

K_full = K_matrix(n=0, ng=NGS[1])
# F_full = apply_bending_moment(M=1000)
F_full = apply_axial_force(N=1000)
K, F = apply_boundary_conditions(K_full, F_full, fixed_dofs=fixed_dofs)
u = solve_system(K, F)
print('Case: axial force')
print("Stresses are:")
sp.pprint(get_stress_vector(u, n=0)[2])

print()

Case: bending moment
Stresses are:
-6.94695855693771
-13.8939171138754

Case: axial force
Stresses are:
7.87911235801553



In [22]:
# Get the list of Gauss points and weights
gauss_points = gi_data[NGS[1]]['point']
gauss_weights = gi_data[NGS[1]]['weight']

print(gauss_points)

# The displacement vector you want to analyze
_u = apply_unit_bending_rotation()

print(_u)

# Get the symbolic B and D matrices
B_sym = B_matrix(1).subs(physical_coords_map)
D = D_matrix()

# Loop through each Gauss point
for i in range(len(gauss_points)):
    for j in range(len(gauss_points)):
        gp_i = gauss_points[i]  # Get the coordinates for the current point
        gp_j = gauss_points[j]  # Get the coordinates for the current point

        # Create the substitution dictionary for THIS point
        gp_data = {R: gp_i, Z: gp_j}

        # Evaluate B at this specific Gauss point
        B_at_gp = B_sym.subs(gp_data).evalf()

        # Calculate the stress at this specific Gauss point
        sigma = D @ (B_at_gp @ _u)

        print(f"--- Stress vector at Gauss point {i+1} ({gp_i}, {gp_j}) ---")
        sp.pprint(sigma)
        print("\n")



[-0.5774, 0.5774]
[[ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [-0.01]
 [ 0.  ]
 [ 0.  ]
 [ 0.01]]
--- Stress vector at Gauss point 1 (-0.5774, -0.5774) ---
⎡666230.769230769 ⎤
⎢                 ⎥
⎢666230.769230769 ⎥
⎢                 ⎥
⎢1554538.46153846 ⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎢-93454.7678434015⎥
⎢                 ⎥
⎣-16253846.1538462⎦


--- Stress vector at Gauss point 1 (-0.5774, 0.5774) ---
⎡666230.769230769 ⎤
⎢                 ⎥
⎢666230.769230769 ⎥
⎢                 ⎥
⎢1554538.46153846 ⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎢-348829.982953577⎥
⎢                 ⎥
⎣-60669230.7692308⎦


--- Stress vector at Gauss point 2 (0.5774, -0.5774) ---
⎡-666230.769230769⎤
⎢                 ⎥
⎢-666230.769230769⎥
⎢                 ⎥
⎢-1554538.46153846⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎢92392.3113727145 ⎥
⎢                 ⎥
⎣-16253846.1538462⎦


--- Stress vector at Gauss point 2 